In [1]:
import json
import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt

In [43]:
# Here we can add all the methods we have/want
models_list = ['clip_baseline',
    'coop_baseline', 'coop_pseudo_baseline',
    'vpt_baseline', 'vpt_pseudo_baseline', 
    'teacher_student',
]

In [98]:
model = models_list[2]
print(model)
filename = f"results/results_model_{model}.json"

data = [json.loads(i) for i in open(filename,'r').readlines()]

coop_pseudo_baseline


In [100]:
std_accuracies = []
gen_accuracies = []
seen_accuracies = []
unseen_accuracies = []
harmonic_accuracies = []

for m in data:
    # Here change and filter data depending on what results we want/
    # For instance, let's assume we want 
    if (
        (m['config']['DATASET_NAME'] == 'DTD')  \
         and (m['config']['VIS_ENCODER'] == 'ViT-L/14') \
         and (m['config']['SPLIT_SEED'] == 500) \
    ):
        std_accuracies.append(m['std_accuracy'])
        gen_accuracies.append(m['gen_accuracy'])
        seen_accuracies.append(m['gen_seen'])
        unseen_accuracies.append(m['gen_unseen'])
        harmonic_accuracies.append(st.hmean([m['gen_seen'], m['gen_unseen']]))

print(f"Mean STD accuracy: {round((np.sum(std_accuracies)/len(std_accuracies))*100, 2)}")
print(f"Std STD accuracy: {np.std(std_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(std_accuracies)-1, loc=np.mean(std_accuracies), scale=st.sem(std_accuracies)) 
print(f"95% STD confidence interval {round((np.sum(std_accuracies)/len(std_accuracies) - interval[0])*100, 2)}")
print('\n')

print(f"Mean SEEN accuracy: {round((np.sum(seen_accuracies)/len(seen_accuracies))*100, 2)}")
print(f"Std SEEN accuracy: {np.std(seen_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(seen_accuracies)-1, loc=np.mean(seen_accuracies), scale=st.sem(seen_accuracies)) 
print(f"95% SEEN confidence interval {round((np.sum(seen_accuracies)/len(seen_accuracies) - interval[0])*100, 2)}")
print('\n')

print(f"Mean UNSEEN accuracy: {round((np.sum(unseen_accuracies)/len(unseen_accuracies))*100,2)}")
print(f"Std UNSEEN accuracy: {np.std(unseen_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(unseen_accuracies)-1, loc=np.mean(unseen_accuracies), scale=st.sem(unseen_accuracies)) 
print(f"95% UNSEEN confidence interval {round((np.sum(unseen_accuracies)/len(unseen_accuracies) - interval[0])*100,2)}")
print('\n')

print(f"Mean HARMONIC accuracy: {round((np.sum(harmonic_accuracies)/len(harmonic_accuracies))*100, 2)}")
print(f"Std HARMONIC accuracy: {np.std(harmonic_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(harmonic_accuracies)-1, loc=np.mean(harmonic_accuracies), scale=st.sem(harmonic_accuracies)) 
print(f"95% HARMONIC confidence interval {round((np.sum(harmonic_accuracies)/len(harmonic_accuracies) - interval[0])*100, 2)}")

Mean STD accuracy: 59.61
Std STD accuracy: 0.0058531409738070556
95% STD confidence interval 0.81


Mean SEEN accuracy: 75.16
Std SEEN accuracy: 0.005097153620377417
95% SEEN confidence interval 0.71


Mean UNSEEN accuracy: 50.53
Std UNSEEN accuracy: 0.0034471315683307813
95% UNSEEN confidence interval 0.48


Mean HARMONIC accuracy: 60.43
Std HARMONIC accuracy: 0.0028647725711470518
95% HARMONIC confidence interval 0.4


In [29]:
unseen_accuracies

[0.3486111111111111,
 0.3486111111111111,
 0.3680555555555556,
 0.3527777777777778,
 0.35555555555555557,
 0.3638888888888889,
 0.3625,
 0.34305555555555556,
 0.3625,
 0.35,
 0.35138888888888886]

In the google sheets, I report the mean and the confidence interval.

In [18]:
import pickle

In [138]:
with open('pseudolabels/RESICS45_ViT-B32_student_iter_4_pseudolabels.pickle', 'rb') as f:
    data = pickle.load(f)

In [139]:
classes = []
with open(f"../development/RESICS45/train.json", "r") as f:
    datas = json.load(f)
    for d in datas["categories"]:
        classes.append(d["name"].replace("_", " "))

In [141]:
for idx, i in enumerate(data['labels']):
    print(classes[i], data['filepaths'][idx].split('/')[-1])